# Подготовка данных для геокодирования

In [29]:
import re

import pandas as pd
from pandas.io.json import json_normalize

from utils.prepared_addresses import drop_one_time, string_handler

In [30]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 100)

In [31]:
def get_splited_string_len(string: str):
    return len(string.split(' '))

In [27]:
df = pd.read_csv('data/export-reestrmkd-25-20191201.csv', sep=';')
df = df[df.formalname_city == 'Владивосток'].reset_index(drop=True)
print('Записей в исходном файле', len(df))

# Удалить не нужные записи
drop_one_time(df)
df.reset_index(inplace=True)

# Создать колонки для дальнейшего заполнения
df['splited_string_len'] = pd.np.NAN
df['prepared_address_v1'] = pd.np.NAN
df['lat'] = pd.np.NAN
df['lon'] = pd.np.NAN
df['raw_class'] = pd.np.NAN
df['raw_type'] = pd.np.NAN
print('Записей после обработки', len(df))

# Подготовка адреса
for index in df.index:
    df.prepared_address_v1[index] = string_handler(df.address[index])

# Разбиение адреса по пробелам и подсчет количества получившихся элементов
for index in df.index:
    df.splited_string_len[index] = get_splited_string_len(df.prepared_address_v1[index])


Записей в исходном файле 3505
Записей после обработки 3492


/Users/dnv/projects/urban_environment_analysis/env/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/dnv/projects/urban_environment_analysis/env/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [32]:
print(df.groupby(['shortname_street'])['id'].size())
print('\n')
print(df.groupby(['splited_string_len'])['prepared_address_v1'].count())
print('\n')
len(df)

shortname_street
Проспект      78
пер           47
пр-кт        275
тер            4
ул          3088
Name: id, dtype: int64


splited_string_len
7.0    2841
8.0     614
9.0      37
Name: prepared_address_v1, dtype: int64




3492

In [33]:
df.to_csv('data/prepared_addresses.csv', index=False)